In [3]:
import torch
import gc

# 1. Borramos variables pesadas si existen
if 'model' in locals(): del model
if 'owl_model' in locals(): del owl_model

# 2. Vaciamos la caché de CUDA
torch.cuda.empty_cache()
gc.collect()

print("🧹 Memoria de la GPU liberada.")

🧹 Memoria de la GPU liberada.


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
import pandas as pd
import os

device = "cuda" if torch.cuda.is_available() else "cpu"

# Usamos la ruta absoluta de tu proyecto
BASE_DIR = os.path.expanduser("~/Documents/HackUDC_2026")
CARPETA_RECORTES = os.path.join(BASE_DIR, "data", "dataset_entrenar_IA")
CARPETA_CATALOGO = os.path.join(BASE_DIR, "data", "images", "products")
MODELO_SALIDA = os.path.join(BASE_DIR, "modelo_experto_inditex.pt")

# Verificación de seguridad antes de empezar
if not os.path.exists(CARPETA_RECORTES):
    print(f"❌ ERROR: No existe la carpeta de recortes en {CARPETA_RECORTES}")
else:
    n_fotos = len([f for f in os.listdir(CARPETA_RECORTES) if f.endswith('.jpg')])
    print(f"✅ Carpeta encontrada. Hay {n_fotos} imágenes listas para entrenar.")

# --- 2. EL DATASET "ESPEJO" ---
class InditexContrastiveDataset(Dataset):
    def __init__(self, carpeta_recortes, carpeta_catalogo, transform=None):
        self.carpeta_recortes = carpeta_recortes
        self.carpeta_catalogo = carpeta_catalogo
        self.transform = transform
        # Listamos los recortes que acabas de generar
        self.lista_recortes = [f for f in os.listdir(carpeta_recortes) if f.endswith('.jpg')]

    def __len__(self):
        return len(self.lista_recortes)

    def __getitem__(self, idx):
        nombre_recorte = self.lista_recortes[idx]
        # Extraemos el ID de producto del nombre (ej: I_476e19..._from_B_...)
        id_producto = nombre_recorte.split('_from_')[0]
        
        ruta_recorte = os.path.join(self.carpeta_recortes, nombre_recorte)
        ruta_catalogo = os.path.join(self.carpeta_catalogo, f"{id_producto}.jpg")
        
        # Cargamos ambas imágenes
        img_real = Image.open(ruta_recorte).convert("RGB")
        try:
            img_estudio = Image.open(ruta_catalogo).convert("RGB")
        except:
            # Si falta la del catálogo, usamos la misma (solo para evitar errores)
            img_estudio = img_real
            
        if self.transform:
            img_real = self.transform(img_real)
            img_estudio = self.transform(img_estudio)
            
        return img_real, img_estudio

# --- 3. PREPARACIÓN ---
transformaciones = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

dataset = InditexContrastiveDataset(CARPETA_RECORTES, CARPETA_CATALOGO, transform=transformaciones)
# Cambiamos batch_size de 32 a 8 o incluso 4 si sigue petando
loader = DataLoader(dataset, batch_size=8, shuffle=True)
# Usamos una ResNet50 como base (muy buena para texturas de ropa)
modelo = models.resnet50(pretrained=True)
# Cambiamos la última capa para que escupa un vector de 512 dimensiones (como CLIP)
modelo.fc = nn.Linear(modelo.fc.in_features, 512)
modelo = modelo.to(device)

criterion = nn.CosineEmbeddingLoss() # Esta pérdida es perfecta para comparar vectores
optimizer = optim.Adam(modelo.parameters(), lr=0.0001)

# --- 4. BUCLE DE ENTRENAMIENTO (¡A por el oro!) ---
print(f"🔥 Empezando entrenamiento con {len(dataset)} pares de imágenes...")
modelo.train()

for epoch in range(5): # Con 5 épocas para empezar sobra
    running_loss = 0.0
    for img_real, img_estudio in loader:
        img_real, img_estudio = img_real.to(device), img_estudio.to(device)
        
        optimizer.zero_grad()
        
        # Sacamos los vectores de ambas fotos
        vec_real = modelo(img_real)
        vec_estudio = modelo(img_estudio)
        
        # Target 1 significa: "haz que estos dos vectores se parezcan"
        target = torch.ones(vec_real.size(0)).to(device)
        
        loss = criterion(vec_real, vec_estudio, target)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f"Epoch {epoch+1}/5 - Loss: {running_loss/len(loader):.4f}")

# Guardamos tu nuevo modelo experto
torch.save(modelo.state_dict(), MODELO_SALIDA)
print(f"✅ ¡Modelo guardado como {MODELO_SALIDA}!")

✅ Carpeta encontrada. Hay 276 imágenes listas para entrenar.


/home/aleixbertranandreu/Documents/HackUDC_2026/.venv/lib64/python3.13/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/aleixbertranandreu/Documents/HackUDC_2026/.venv/lib64/python3.13/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


🔥 Empezando entrenamiento con 276 pares de imágenes...
Epoch 1/5 - Loss: 0.0301
Epoch 2/5 - Loss: 0.0031
Epoch 3/5 - Loss: 0.0017
Epoch 4/5 - Loss: 0.0011
Epoch 5/5 - Loss: 0.0008
✅ ¡Modelo guardado como /home/aleixbertranandreu/Documents/HackUDC_2026/modelo_experto_inditex.pt!


In [2]:
import torch
import torch.nn as nn
from torchvision import transforms, models
from PIL import Image
import os
import pandas as pd
from tqdm import tqdm

# --- 1. CONFIGURACIÓN ---
device = "cuda" if torch.cuda.is_available() else "cpu"
BASE_DIR = os.path.expanduser("~/Documents/HackUDC_2026")
CARPETA_PRODUCTOS = os.path.join(BASE_DIR, "data", "images", "products")
RUTA_MODELO = os.path.join(BASE_DIR, "modelo_experto_inditex.pt")

# --- 2. CARGAR TU MODELO EXPERTO ---
modelo = models.resnet50()
modelo.fc = nn.Linear(modelo.fc.in_features, 512)
modelo.load_state_dict(torch.load(RUTA_MODELO))
modelo = modelo.to(device)
modelo.eval() # Modo "solo lectura"

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# --- 3. GENERAR LA GALERÍA ---
nombres_productos = []
embeddings_lista = []

print("🛰️ Generando base de datos de huellas dactilares del catálogo...")
fotos_catalogo = [f for f in os.listdir(CARPETA_PRODUCTOS) if f.endswith('.jpg')]

with torch.no_grad():
    for nombre in tqdm(fotos_catalogo):
        ruta = os.path.join(CARPETA_PRODUCTOS, nombre)
        try:
            img = Image.open(ruta).convert("RGB")
            img_t = transform(img).unsqueeze(0).to(device)
            
            # Sacamos el vector de 512 dimensiones
            vector = modelo(img_t)
            
            embeddings_lista.append(vector.cpu())
            nombres_productos.append(nombre.replace(".jpg", ""))
        except:
            continue

# Guardamos todo en un solo bloque para que la búsqueda sea instantánea
galeria_vectores = torch.cat(embeddings_lista)
torch.save({'vectores': galeria_vectores, 'ids': nombres_productos}, "galeria_expert.pt")

print(f"✅ Galería guardada con {len(nombres_productos)} productos lista para buscar.")

🛰️ Generando base de datos de huellas dactilares del catálogo...


100%|██████████| 5604/5604 [03:58<00:00, 23.54it/s]

✅ Galería guardada con 5604 productos lista para buscar.


In [4]:
import torch
import torch.nn as nn
from torchvision import transforms, models
from PIL import Image
import os
from tqdm import tqdm

# --- 1. CONFIGURACIÓN ---
device = "cuda" if torch.cuda.is_available() else "cpu"
BASE_DIR = os.path.expanduser("~/Documents/HackUDC_2026")
CARPETA_PRODUCTOS = os.path.join(BASE_DIR, "data/images/products")
RUTA_MODELO = os.path.join(BASE_DIR, "modelo_experto_inditex.pt")
# 🔥 RUTA ABSOLUTA PARA QUE NO SE PIERDA 🔥
RUTA_GALERIA_SALIDA = os.path.join(BASE_DIR, "galeria_expert.pt")

# --- 2. CARGAR TU MODELO EXPERTO ---
modelo = models.resnet50()
modelo.fc = nn.Linear(modelo.fc.in_features, 512)
modelo.load_state_dict(torch.load(RUTA_MODELO, map_location=device))
modelo = modelo.to(device)
modelo.eval()

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

nombres_productos = []
embeddings_lista = []

print(f"🛰️ Generando galería en: {RUTA_GALERIA_SALIDA}")
fotos_catalogo = [f for f in os.listdir(CARPETA_PRODUCTOS) if f.endswith('.jpg')]

with torch.no_grad():
    for nombre in tqdm(fotos_catalogo):
        ruta = os.path.join(CARPETA_PRODUCTOS, nombre)
        try:
            img = Image.open(ruta).convert("RGB")
            img_t = transform(img).unsqueeze(0).to(device)
            vector = modelo(img_t)
            embeddings_lista.append(vector.cpu())
            nombres_productos.append(nombre.replace(".jpg", ""))
        except:
            continue

# --- 3. GUARDAR ---
if len(embeddings_lista) > 0:
    galeria_vectores = torch.cat(embeddings_lista)
    torch.save({'vectores': galeria_vectores, 'ids': nombres_productos}, RUTA_GALERIA_SALIDA)
    print(f"✅ ¡LOGRADO! Galería guardada con {len(nombres_productos)} productos.")
else:
    print("❌ ERROR: No se encontraron fotos en la carpeta de productos.")

🛰️ Generando galería en: /home/aleixbertranandreu/Documents/HackUDC_2026/galeria_expert.pt


100%|██████████| 5604/5604 [04:16<00:00, 21.84it/s]

✅ ¡LOGRADO! Galería guardada con 5604 productos.


📂 Archivos en tu carpeta raíz:
 -> modelo_experto_inditex.pt
 -> best.pt


In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, models
from transformers import OwlViTProcessor, OwlViTForObjectDetection
from PIL import Image
import os
import pandas as pd
from tqdm import tqdm

# --- 1. CONFIGURACIÓN FINAL ---
device = "cuda" if torch.cuda.is_available() else "cpu"
BASE_DIR = os.path.expanduser("~/Documents/HackUDC_2026")
RUTA_MODELO = os.path.join(BASE_DIR, "modelo_experto_inditex.pt")
RUTA_GALERIA = os.path.join(BASE_DIR, "galeria_expert.pt")
CARPETA_BUNDLES = os.path.join(BASE_DIR, "data/images/bundles")
RUTA_TEST_CSV = os.path.join(BASE_DIR, "data/raw/bundles_product_match_test.csv")
ARCHIVO_SALIDA = os.path.join(BASE_DIR, "submission_chiringuito.csv")

print(f"🚀 Iniciando generación de submission en {device}...")

# --- 2. CARGAR MODELOS Y GALERÍA ---
processor = OwlViTProcessor.from_pretrained("google/owlvit-base-patch32")
owl_model = OwlViTForObjectDetection.from_pretrained("google/owlvit-base-patch32").to(device)

expert_model = models.resnet50()
expert_model.fc = nn.Linear(expert_model.fc.in_features, 512)
expert_model.load_state_dict(torch.load(RUTA_MODELO, map_location=device))
expert_model = expert_model.to(device).eval()

data_galeria = torch.load(RUTA_GALERIA, map_location=device)
galeria_vectores = F.normalize(data_galeria['vectores'].to(device), p=2, dim=1)
galeria_ids = data_galeria['ids']

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# --- 3. PROCESAR EL TEST SET ---
df_test = pd.read_csv(RUTA_TEST_CSV)
bundles_test = df_test['bundle_asset_id'].unique()

resultados = []

print(f"📑 Procesando {len(bundles_test)} bundles para la entrega final...")

with torch.no_grad():
    for bundle_id in tqdm(bundles_test):
        ruta_img = os.path.join(CARPETA_BUNDLES, f"{bundle_id}.jpg")
        if not os.path.exists(ruta_img):
            # Si no hay foto, ponemos 15 IDs vacíos o aleatorios (mejor que nada)
            resultados.append([bundle_id] + [""] * 15)
            continue
            
        img = Image.open(ruta_img).convert("RGB")
        
        # Intentamos detectar CUALQUIER prenda (upper o lower)
        etiquetas = [["upper clothing", "lower clothing", "dress"]]
        inputs = processor(text=etiquetas, images=img, return_tensors="pt").to(device)
        outputs = owl_model(**inputs)
        
        target_sizes = torch.tensor([img.size[::-1]])
        dets = processor.post_process_grounded_object_detection(
            outputs=outputs, target_sizes=target_sizes, text_labels=etiquetas, threshold=0.1
        )[0]
        
        if len(dets["boxes"]) > 0:
            # Cogemos la detección más segura
            box = dets["boxes"][0]
            x1, y1, x2, y2 = map(int, box.tolist())
            recorte = img.crop((x1, y1, x2, y2))
            
            # Sacamos embedding
            rec_t = transform(recorte).unsqueeze(0).to(device)
            query_emb = F.normalize(expert_model(rec_t), p=2, dim=1)
            
            # Buscamos en galería
            sims = torch.mm(query_emb, galeria_vectores.t())
            _, indices = torch.topk(sims, 15)
            
            top_15 = [galeria_ids[idx] for idx in indices[0]]
        else:
            # Si no detecta nada, devolvemos los 15 primeros de la galería (fail-safe)
            top_15 = galeria_ids[:15]
            
        resultados.append([bundle_id] + top_15)

# --- 4. GUARDAR RESULTADOS ---
columnas = ['bundle_asset_id'] + [f'product_asset_id_{i}' for i in range(1, 16)]
df_final = pd.DataFrame(resultados, columns=columnas)
df_final.to_csv(ARCHIVO_SALIDA, index=False)

print(f"\n🏆 ¡SUBMISSION GENERADA! Archivo listo en: {ARCHIVO_SALIDA}")

🚀 Iniciando generación de submission en cuda...


Loading weights:   0%|          | 0/412 [00:00<?, ?it/s]

OwlViTForObjectDetection LOAD REPORT from: google/owlvit-base-patch32
Key                                         | Status     |  | 
--------------------------------------------+------------+--+-
owlvit.vision_model.embeddings.position_ids | UNEXPECTED |  | 
owlvit.text_model.embeddings.position_ids   | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


📑 Procesando 455 bundles para la entrega final...


100%|██████████| 455/455 [00:51<00:00,  8.78it/s]


🏆 ¡SUBMISSION GENERADA! Archivo listo en: /home/aleixbertranandreu/Documents/HackUDC_2026/submission_chiringuito.csv


In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, models
from transformers import OwlViTProcessor, OwlViTForObjectDetection
from PIL import Image
import os
import pandas as pd
from tqdm import tqdm

# --- 1. CONFIGURACIÓN ---
device = "cuda" if torch.cuda.is_available() else "cpu"
BASE_DIR = os.path.expanduser("~/Documents/HackUDC_2026")
RUTA_MODELO = os.path.join(BASE_DIR, "modelo_experto_inditex.pt")
RUTA_GALERIA = os.path.join(BASE_DIR, "galeria_expert.pt")
CARPETA_BUNDLES = os.path.join(BASE_DIR, "data/images/bundles")
# Usamos el archivo de TEST que es el que hay que predecir
RUTA_TEST_CSV = os.path.join(BASE_DIR, "data/raw/bundles_product_match_test.csv")
ARCHIVO_SALIDA = os.path.join(BASE_DIR, "submission.csv")

# --- 2. CARGA DE MODELOS ---
print("⏳ Cargando motores...")
processor = OwlViTProcessor.from_pretrained("google/owlvit-base-patch32")
owl_model = OwlViTForObjectDetection.from_pretrained("google/owlvit-base-patch32").to(device)

expert_model = models.resnet50()
expert_model.fc = nn.Linear(expert_model.fc.in_features, 512)
expert_model.load_state_dict(torch.load(RUTA_MODELO, map_location=device))
expert_model = expert_model.to(device).eval()

data_galeria = torch.load(RUTA_GALERIA, map_location=device)
galeria_vectores = F.normalize(data_galeria['vectores'].to(device), p=2, dim=1)
galeria_ids = data_galeria['ids']

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# --- 3. PROCESADO ---
df_test = pd.read_csv(RUTA_TEST_CSV)
bundles_test = df_test['bundle_asset_id'].unique()

rows = [] # Aquí guardaremos parejas [bundle, producto]

print(f"🏃‍♂️ Procesando {len(bundles_test)} bundles para el formato oficial...")

with torch.no_grad():
    for bundle_id in tqdm(bundles_test):
        ruta_img = os.path.join(CARPETA_BUNDLES, f"{bundle_id}.jpg")
        
        # Si no existe la foto, metemos 15 vacíos para no romper el CSV
        if not os.path.exists(ruta_img):
            for _ in range(15):
                rows.append({'bundle_asset_id': bundle_id, 'product_asset_id': ""})
            continue
            
        img = Image.open(ruta_img).convert("RGB")
        
        # Detección rápida
        etiquetas = [["upper clothing", "lower clothing", "dress"]]
        inputs = processor(text=etiquetas, images=img, return_tensors="pt").to(device)
        outputs = owl_model(**inputs)
        
        target_sizes = torch.tensor([img.size[::-1]])
        dets = processor.post_process_grounded_object_detection(
            outputs=outputs, target_sizes=target_sizes, text_labels=etiquetas, threshold=0.1
        )[0]
        
        if len(dets["boxes"]) > 0:
            box = dets["boxes"][0]
            x1, y1, x2, y2 = map(int, box.tolist())
            recorte = img.crop((x1, y1, x2, y2))
            
            rec_t = transform(recorte).unsqueeze(0).to(device)
            query_emb = F.normalize(expert_model(rec_t), p=2, dim=1)
            
            sims = torch.mm(query_emb, galeria_vectores.t())
            _, indices = torch.topk(sims, 15)
            top_15 = [galeria_ids[idx] for idx in indices[0]]
        else:
            # Fallback si no detecta nada
            top_15 = galeria_ids[:15]
            
        # 🌟 LA CLAVE: Añadimos una fila por cada producto
        for prod_id in top_15:
            rows.append({'bundle_asset_id': bundle_id, 'product_asset_id': prod_id})

# --- 4. GUARDAR ---
df_final = pd.DataFrame(rows)
df_final.to_csv(ARCHIVO_SALIDA, index=False)

print(f"\n✅ ¡SUBMISSION LISTA! Formato de {len(df_final)} filas generado en: {ARCHIVO_SALIDA}")

⏳ Cargando motores...


Loading weights:   0%|          | 0/412 [00:00<?, ?it/s]

OwlViTForObjectDetection LOAD REPORT from: google/owlvit-base-patch32
Key                                         | Status     |  | 
--------------------------------------------+------------+--+-
owlvit.vision_model.embeddings.position_ids | UNEXPECTED |  | 
owlvit.text_model.embeddings.position_ids   | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


🏃‍♂️ Procesando 455 bundles para el formato oficial...


100%|██████████| 455/455 [00:46<00:00,  9.86it/s]


✅ ¡SUBMISSION LISTA! Formato de 6825 filas generado en: /home/aleixbertranandreu/Documents/HackUDC_2026/submission.csv
